In [43]:
import pandas 
import numpy

#TODO Add variable that decides if we stop at first full_quantity match solution, or search further
#TODO Add overstoch variable

# HYPERSPACE
# the data is sorted by time
# random data
data = [
    [1, 1, "RFNS", 20],
    [2, 1, "RFPS", 100],
    [3, 3, "RFPS", 55],
    [4, 4, "RFNS", 60],
    [5, 5, "RFNS", 25],
    [6, 6, "RFNS", 30],
    [7, 7, "RFNS", 20]
]
# perfect FIFO
# data = [
#     [1, 1, "RFNS", 100],
#     [2, 1, "RFPS", 100],
#     [3, 3, "RFPS", 50],
#     [4, 3, "RFPS", 50],
#     [5, 5, "RFNS", 50],
#     [6, 6, "RFNS", 25],
#     [7, 7, "RFNS", 25]
# ]
df = pandas.DataFrame(data, columns=["TRANSACTION_ID", "TIMESTAMP", "ACTION", "NET_QUANTITY"])

backflushing_penalty = -1000
link_profit = 1
action_that_puts = "RFPS"
action_that_takes = "RFNS"
action_column = "ACTION"
quantity_column = "NET_QUANTITY"
allowed_x_values = [0, 1]

print(df)

   TRANSACTION_ID  TIMESTAMP ACTION  NET_QUANTITY
0               1          1   RFNS            20
1               2          1   RFPS           100
2               3          3   RFPS            55
3               4          4   RFNS            60
4               5          5   RFNS            25
5               6          6   RFNS            30
6               7          7   RFNS            20


In [44]:
# Prepare the data
put_df = df[df[action_column] == action_that_puts].reset_index(drop=True)
take_df = df[df[action_column] == action_that_takes].reset_index(drop=True)
nb_puts = len(put_df.index)
nb_takes = len(take_df.index)

# profit matrix
profits = numpy.full((nb_puts, nb_takes), link_profit, dtype=int)
# Find backflushing examples and correct the profit matrix with backflushing penalties
for put_id, row in put_df.iterrows():
    
    put_timestamp = row["TIMESTAMP"]
    backflushing_take_ids = list(take_df[take_df["TIMESTAMP"] < put_timestamp].index.values)
    
    for take_id in backflushing_take_ids:
        profits[put_id][take_id] = backflushing_penalty

# quantities
put_q = numpy.array(put_df[quantity_column].values)
take_q = numpy.array(take_df[quantity_column].values)

# choose initial solution that will be the optimum in case of perfect FIFO
put_cumulative_q = numpy.cumsum(put_q)
take_cumulative_q = numpy.cumsum(take_q)
x = numpy.zeros((nb_puts, nb_takes), dtype=int)
for t_i, t_q in enumerate(take_cumulative_q):
    for p_i, p_q in enumerate(put_cumulative_q):
        if p_q >= t_q:
            x[p_i][t_i] = 1
            break

print(f"put df: {put_df}")
print(f"take df: {take_df}")
print(f"number of puts: {nb_puts}")
print(f"number of takes: {nb_takes}")
print(f"profits: {profits}")
print(f"put quantities: {put_q}")
print(f"take quantities: {take_q}")
print(f"put cumul quantities: {put_cumulative_q}")
print(f"take cumul quantities: {take_cumulative_q}")
print(f"x: {x}")


put df:    TRANSACTION_ID  TIMESTAMP ACTION  NET_QUANTITY
0               2          1   RFPS           100
1               3          3   RFPS            55
take df:    TRANSACTION_ID  TIMESTAMP ACTION  NET_QUANTITY
0               1          1   RFNS            20
1               4          4   RFNS            60
2               5          5   RFNS            25
3               6          6   RFNS            30
4               7          7   RFNS            20
number of puts: 2
number of takes: 5
profits: [[    1     1     1     1     1]
 [-1000     1     1     1     1]]
put quantities: [100  55]
take quantities: [20 60 25 30 20]
put cumul quantities: [100 155]
take cumul quantities: [ 20  80 105 135 155]
x: [[1 1 0 0 0]
 [0 0 1 1 1]]


In [58]:

# inequality constraints 
# (Note: limits of the decision variable can be integrated in the generation of a new solution)
# (Source: https://programmer.ink/think/processing-of-python-programming-constraints-of-simulated-annealing-algorithm.html)
def _test_quantity_constraint(x):
    """
    Compare the linked take quantities with the put quantities they come from.
    Linked quantities should never be greater than their corresponding put quantity.

    In principle, we need a full match and only the full match will optimize the objective ftn,
    but to limit calculation times, it may be interesting to accept non-optimal solutions.

    :param x:           the proposed solution
    :return viable:     boolean variable that indicates if the solution is viable
    """
    linked_quantities = numpy.matmul(x, take_q)

    return numpy.less_equal(linked_quantities, put_q).all():

def _test_full_quantity_match(x):
    """
    Compare the linked take quantities with the put quantities they come from.
    If the linked take quantities are equal to to their corresponding put quantity,
    we have a full quantity match.

    :param x:           the proposed solution
    :return full_match: boolean variable that indicates if the solution is perfect
    """
    linked_quantities = numpy.matmul(x, take_q)

    return numpy.array_equal(linked_quantities, put_q)


Test constraints on initial solution: viable = False, full quantity match = False


In [ ]:
# CREATE ONE VIABLE SOLUTION - Simulated annealing
def _generate_viable_solution(x):

    #TODO calculate some new solution
    x_new = None

    # test the solution
    viable_solution = _test_quantity_constraint(x_new)

    if not viable_solution:
        _generate_viable_solution(x_new)
    
    return x_new
    
def _simulated_annealing(x_old):

    # this is the core algotithm

    #TODO Create a viable solution with the previous function
    x_new = _generate_viable_solution(x_old)
    #(Not sure abt this one)TODO In case we have a perfect quantity match, return the solution and end the algo
    if _test_full_quantity_match(x_new):
        return x_new
    #TODO Calculate profit
    #TODO Follow simulated annealing principles to choose between solutions and repeat the loop

def apply():
    return None


In [ ]:
# SIMULATED ANNEALING
# Iteratively create viable solutions with the above function, and then find the one that optimizes a goal function